### test_data (pickle檔) 111619 rows × 6 columns
欄位 review_index / userId / appId / roberta_embedding / bert_embedding / rating

### train_data_dd.csv 904120 rows × 6 columns
欄位 review_index / userId / appId / roberta_embedding / bert_embedding / rating

### val_data (pickle檔) 100457 rows × 6 columns
欄位 review_index / userId / appId / roberta_embedding / bert_embedding / rating

test_index.txt  欄位：review_index、userId、appId ； Shape：111619 rows × 3 columns

train_index.txt  欄位：review_index、userId、appId； Shape：904120 rows × 3 columns 不含val

val_index.txt  欄位：review_index、userId、appId； Shape：100457 rows × 3 columns

In [1]:
import pickle
import pandas as pd
import numpy as np
import time
import os
import re
from tqdm.notebook import tqdm
import csv
from pandas.core.frame import DataFrame

### 讀入完整 rating

In [2]:
# ALL_ORIGIN_DATA_rating.csv 僅:index/評分/appID/userID

ALL_ORIGIN_DATA_rating = pd.read_csv('./Data/ALL_ORIGIN_DATA_rating.csv')
ALL_ORIGIN_DATA_rating.head()

,Unnamed: 0,review_index,userId,appId,rating
0,0,0,0,5,1
1,1,1,1,5,3
2,2,2,2,5,5
3,3,3,3,5,5
4,4,4,4,5,5


### 讀入完整roberta_emb

In [3]:
#讀入pickle
df_roberta = open('./Data/ALL_ORIGIN_DATA_review_roberta_0toend_embedding','rb')#注意此处是rb
roberta=pickle.load(df_roberta) # 讀入用rb
df_roberta.close() 

In [4]:
roberta.head()

,roberta_embedding,Unnamed: 0
0,"[-0.05433040112257004, 0.04897385835647583, -0...",NaN
1,"[-0.030177436769008636, 0.04714367538690567, -...",NaN
2,"[-0.059469737112522125, 0.0544145405292511, -0...",NaN
3,"[-0.048043347895145416, 0.009005382657051086, ...",NaN
4,"[-0.027240321040153503, 0.051726289093494415, ...",NaN


### 讀入完整bert_emb

In [13]:
ALL_ORIGIN_DATA_review_bert = pd.read_csv('./Data/ALL_ORIGIN_DATA_review_bert.csv')

### 切出test set:

In [5]:
test_index = pd.read_csv('./Data/test_index.txt')
print(test_index[:5],'\n','\n')

   review_index  userId  appId
0        556989  104298      5
1        276457   24488      5
2        827705  100603      5
3        292180   56006      5
4        794402  109560      5 
 



In [6]:
len(test_index)

111619

###  

In [14]:
type(ALL_ORIGIN_DATA_review_bert['bert_embedding'][0])

str

In [7]:
type(roberta['roberta_embedding'][0]) #list

list

In [8]:
# 找出所有test_index 對應的 rating
rating_lst=[] #準備存進去 text_index的 xlnet_emb lst
for count,i in tqdm(enumerate(test_index['review_index'])):
    rating_lst.append(ALL_ORIGIN_DATA_rating['rating'][i])

0it [00:00, ?it/s]

In [9]:
type(rating_lst) #list

list

In [15]:
# 找出所有test_index 對應的 bert_embedding
bert_emb_lst=[] #準備存進去 text_index的 bert_emb lst
for count,i in tqdm(enumerate(test_index['review_index'])):
    bert_emb_lst.append(ALL_ORIGIN_DATA_review_bert['bert_embedding'][i])

0it [00:00, ?it/s]

In [16]:
len(bert_emb_lst[0])

16258

In [10]:
# 找出所有test_index 對應的 roberta_embedding
ro_emb_lst=[] #準備存進去 text_index的 ro_emb lst
for count,i in tqdm(enumerate(test_index['review_index'])):
    ro_emb_lst.append(roberta['roberta_embedding'][i])

0it [00:00, ?it/s]

In [11]:
len(ro_emb_lst)

111619

In [12]:
len(rating_lst)

111619

In [17]:
dic_test ={'review_index' : list(test_index['review_index']),"userId" : list(test_index['userId']),"appId":list(test_index['appId']),"roberta_embedding" : ro_emb_lst,"bert_embedding" : bert_emb_lst,"rating":rating_lst }
df_test = DataFrame(dic_test)
df_test

,review_index,userId,appId,roberta_embedding,bert_embedding,rating
0,556989,104298,5,"[-0.06583990901708603, 0.07123593986034393, -0...","[0.06709515303373337, -0.06524760276079178, 0....",5
1,276457,24488,5,"[-0.04401491582393646, 0.06369167566299438, -0...","[0.018158985301852226, 0.2492251843214035, 0.1...",5
2,827705,100603,5,"[-0.020159393548965454, 0.08451575040817261, -...","[-0.12789605557918549, -0.011563075706362724, ...",2
3,292180,56006,5,"[-0.03614839166402817, 0.08437591791152954, -0...","[-0.13007041811943054, -0.20364640653133392, 0...",5
4,794402,109560,5,"[-0.02333754301071167, 0.07442940771579742, -0...","[0.03204464539885521, 0.034370727837085724, 0....",2
...,...,...,...,...,...,...
111614,762316,108249,1110,"[-0.027890361845493317, 0.08228614181280136, -...","[-0.045029934495687485, -0.29239487648010254, ...",3
111615,3507,3332,1110,"[-0.05888579040765762, 0.07512751966714859, 0....","[-0.04986397549510002, -0.5851753354072571, 0....",4
111616,789357,115038,1110,"[-0.0563468262553215, 0.031397443264722824, 0....","[0.21711109578609467, -0.235995352268219, 1.07...",5
111617,112901,74020,1110,"[-0.03713545203208923, 0.0828799456357956, -0....","[0.5416073203086853, 0.07227365672588348, 0.44...",5


In [18]:
import pickle
with open('./Data/test_data', 'wb') as f:
    pickle.dump(df_test, f)

### 切出train set:

In [22]:
train_index = pd.read_csv('./Data/train_index.txt')
print(train_index[:5],'\n','\n')

   review_index  userId  appId
0             0       0      5
1             1       1      5
2             2       2      5
3             3       3      5
4             4       4      5 
 



In [23]:
len(train_index)

904120

In [24]:
# 找出所有 train_index 對應的 rating
rating_lst=[] #準備存進去 train_index 的 rating
for count,i in tqdm(enumerate(train_index['review_index'])):
    rating_lst.append(ALL_ORIGIN_DATA_rating['rating'][i])

0it [00:00, ?it/s]

In [25]:
# 找出所有 train_index 對應的 bert_embedding
bert_emb_lst=[] #準備存進去 train_index 的 bert_emb lst
for count,i in tqdm(enumerate(train_index['review_index'])):
    bert_emb_lst.append(ALL_ORIGIN_DATA_review_bert['bert_embedding'][i])

0it [00:00, ?it/s]

In [26]:
# 找出所有 train_index 對應的 roberta_embedding
ro_emb_lst=[] #準備存進去 train_index 的 ro_emb lst
for count,i in tqdm(enumerate(train_index['review_index'])):
    ro_emb_lst.append(roberta['roberta_embedding'][i])

0it [00:00, ?it/s]

In [27]:
dic_train ={'review_index' : list(train_index['review_index']),"userId" : list(train_index['userId']),"appId":list(train_index['appId']),"roberta_embedding" : ro_emb_lst,"bert_embedding" : bert_emb_lst,"rating": rating_lst }
df_train = DataFrame(dic_train)
df_train.head()

,review_index,userId,appId,roberta_embedding,bert_embedding,rating
0,0,0,5,"[-0.05433040112257004, 0.04897385835647583, -0...","[-0.10288368165493011, -0.019872549921274185, ...",1
1,1,1,5,"[-0.030177436769008636, 0.04714367538690567, -...","[0.006627775263041258, -0.5326172709465027, 0....",3
2,2,2,5,"[-0.059469737112522125, 0.0544145405292511, -0...","[0.023161377757787704, -0.36857113242149353, 0...",5
3,3,3,5,"[-0.048043347895145416, 0.009005382657051086, ...","[0.028108451515436172, -0.30558472871780396, 0...",5
4,4,4,5,"[-0.027240321040153503, 0.051726289093494415, ...","[0.5087504386901855, 0.16101887822151184, 0.62...",5


In [28]:
from dask import dataframe as dd

In [29]:
dd_train = dd.from_pandas(df_train, chunksize=10000)

In [30]:
# 存 train_data
dd_train.to_csv('./Data/train_data_dd.csv',index=False)

['D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\00.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\01.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\02.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\03.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\04.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\05.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\06.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\07.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\08.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\09.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\10.part',
 'D:/pytorch/GoogleGame_RecommenderSystem-main/Data/train_data_dd.csv\\11.part',
 'D:/pytorch/GoogleGame_Reco

### val set

In [31]:
# 把 train_index.txt 也讀進來 

val_index = pd.read_csv('./Data/val_index.txt')
print(val_index[:5],'\n','\n')

   review_index  userId  appId
0        800602   30304   6378
1        870651  103296   6998
2        340076   26498   3216
3         74159   53877    701
4        549663   29013   3920 
 



In [32]:
len(val_index)

100457

In [33]:
# 找出所有 val_index 對應的 rating
rating_lst=[] #準備存進去 text_index的 xlnet_emb lst
for count,i in tqdm(enumerate(val_index['review_index'])):
    rating_lst.append(ALL_ORIGIN_DATA_rating['rating'][i])

0it [00:00, ?it/s]

In [34]:
# 找出所有 val_index 對應的 bert_embedding
bert_emb_lst=[] #準備存進去 val_index 的 bert_emb lst
for count,i in tqdm(enumerate(val_index['review_index'])):
    bert_emb_lst.append(ALL_ORIGIN_DATA_review_bert['bert_embedding'][i])

0it [00:00, ?it/s]

In [35]:
# 找出所有 val_index 對應的 roberta_embedding
ro_emb_lst=[] #準備存進去 val_index 的 ro_emb lst
for count,i in tqdm(enumerate(val_index['review_index'])):
    ro_emb_lst.append(roberta['roberta_embedding'][i])

0it [00:00, ?it/s]

In [36]:
dic_val ={'review_index' : list(val_index['review_index']),"userId" : list(val_index['userId']),"appId":list(val_index['appId']),"roberta_embedding" : ro_emb_lst,"bert_embedding" : bert_emb_lst ,"rating": rating_lst }
df_val = DataFrame(dic_val)
df_val

,review_index,userId,appId,roberta_embedding,bert_embedding,rating
0,800602,30304,6378,"[-0.026093758642673492, 0.07183625549077988, -...","[0.18903899192810059, -0.20202893018722534, 0....",1
1,870651,103296,6998,"[-0.02652128040790558, 0.08051957190036774, -0...","[-0.08873194456100464, -0.6624205112457275, 0....",5
2,340076,26498,3216,"[-0.045878589153289795, 0.0483371764421463, -0...","[0.3070529103279114, -0.3259717524051666, 0.56...",1
3,74159,53877,701,"[-0.0389552041888237, 0.08836665749549866, -0....","[0.28427016735076904, -0.002038797130808234, 0...",5
4,549663,29013,3920,"[-0.053048275411129, 0.030511289834976196, -0....","[0.08091507852077484, -0.02626938372850418, 0....",3
...,...,...,...,...,...,...
100452,700298,127337,5377,"[-0.04661758244037628, 0.08716677129268646, -0...","[0.15324874222278595, 0.1666506826877594, 0.28...",5
100453,219702,56862,2148,"[-0.0392649844288826, 0.0929616242647171, -0.0...","[-0.10891252756118774, -0.36415526270866394, 0...",5
100454,643323,37277,4791,"[-0.04726783186197281, 0.055477265268564224, -...","[-0.1152479276061058, 0.2889108657836914, 0.02...",4
100455,621739,48691,4718,"[0.0076506733894348145, 0.11311863362789154, -...","[-0.08982546627521515, -0.036317404359579086, ...",4


In [37]:
# 存 val_data
import pickle
with open('./Data/val_data', 'wb') as f:
    pickle.dump(df_val, f)